# Ejercicio_4_ID_MAP

## **Introducción:**

En este punto, usaremos el problema de predicción utilizando la base de datos "**Open Problems– Single-Cell Perturbations**", la cual contiene datos experimentales sobre perturbaciones en células individuales. Para ello, utilizaremos técnicas de machine learning, incluyendo K-NN (lineal regresion) y regresión lineal, con el fin de identificar el modelo que mejor se ajuste a los datos.

Finalmente, evaluaremos el desempeño de los modelos construidos mediante la creación de una tabla de error que incluirá las métricas más relevantes en problemas de regresión: MAPE, MAE, RMSE, MSE y R². Esta tabla nos permitirá comparar la eficacia de los modelos.

# Librerias

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import mean_squared_error,make_scorer, mean_absolute_percentage_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import make_scorer, mean_squared_error, r2_score


# Base de datos

En esta sección, cargamos la base de datos que utilizaremos para nuestros modelos: K-NN (lineal regresion) y regresión lineal. Es importante tener en cuenta que el análisis descriptivo de esta variable se encuentra desarrollado en el punto 2, inciso 2, del examen práctico.

In [2]:
'''# Ruta del archivo Parquet
file_path = 'https://drive.google.com/uc?export=download&id=1Jwi-8080GYIStM3Wp6AyVQ-KmQsXr0YO'
df_parqdtra = pd.read_parquet(file_path, engine='pyarrow')
'''

"# Ruta del archivo Parquet\nfile_path = 'https://drive.google.com/uc?export=download&id=1Jwi-8080GYIStM3Wp6AyVQ-KmQsXr0YO'\ndf_parqdtra = pd.read_parquet(file_path, engine='pyarrow')\n"

In [3]:
'''print(df_parqdtra.head())
print(df_parqdtra.size)'''

'print(df_parqdtra.head())\nprint(df_parqdtra.size)'

```
            cell_type             sm_name sm_lincs_id  \
0            NK cells        Clotrimazole    LSM-5341   
1        T cells CD4+        Clotrimazole    LSM-5341   
2        T cells CD8+        Clotrimazole    LSM-5341   
3  T regulatory cells        Clotrimazole    LSM-5341   
4            NK cells  Mometasone Furoate    LSM-3349   

                                              SMILES  control      A1BG  \
0             Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1    False  0.104720   
1             Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1    False  0.915953   
2             Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1    False -0.387721   
3             Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1    False  0.232893   
4  C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...    False  4.290652   

   A1BG-AS1       A2M   A2M-AS1     A2MP1  ...      ZUP1      ZW10    ZWILCH  \
0 -0.077524 -1.625596 -0.144545  0.143555  ... -0.227781 -0.010752 -0.023881   
1 -0.884380  0.371834 -0.081677 -0.498266  ... -0.494985 -0.303419  0.304955   
2 -0.305378  0.567777  0.303895 -0.022653  ... -0.119422 -0.033608 -0.153123   
3  0.129029  0.336897  0.486946  0.767661  ...  0.451679  0.704643  0.015468   
4 -0.063864 -0.017443 -0.541154  0.570982  ...  0.758474  0.510762  0.607401   

      ZWINT      ZXDA      ZXDB      ZXDC    ZYG11B       ZYX     ZZEF1  
0  0.674536 -0.453068  0.005164 -0.094959  0.034127  0.221377  0.368755  
1 -0.333905 -0.315516 -0.369626 -0.095079  0.704780  1.096702 -0.869887  
2  0.183597 -0.555678 -1.494789 -0.213550  0.415768  0.078439 -0.259365  
...
4 -0.123059  0.214366  0.487838 -0.819775  0.112365 -0.122193  0.676629  

[5 rows x 18216 columns]
11184624
```

# Pipeline y Gridsearchcv()

In [4]:
'''import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Crea una copia del DataFrame original
df_parqdtra_copy = df_parqdtra.copy()

# Aplicar One-Hot Encoding a 'cell_type' y 'sm_name'
encoder = OneHotEncoder(sparse=False, drop='first')  # drop='first' evita la multicolinealidad
encoded_features = encoder.fit_transform(df_parqdtra_copy[['cell_type', 'sm_name']])

# Crear un DataFrame con las columnas codificadas
df_encoded = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(['cell_type', 'sm_name']))

# Concatenar las columnas codificadas con el resto del DataFrame (excluyendo 'cell_type' y 'sm_name')
df_parqdtra_encoded = pd.concat([df_parqdtra_copy.drop(columns=['cell_type', 'sm_name']), df_encoded], axis=1)

X = df_encoded
# Las columnas de genes
y = df_parqdtra_encoded[df_parqdtra.columns[4:]]
'''

"import pandas as pd\nfrom sklearn.preprocessing import OneHotEncoder\n\n# Crea una copia del DataFrame original\ndf_parqdtra_copy = df_parqdtra.copy()\n\n# Aplicar One-Hot Encoding a 'cell_type' y 'sm_name'\nencoder = OneHotEncoder(sparse=False, drop='first')  # drop='first' evita la multicolinealidad\nencoded_features = encoder.fit_transform(df_parqdtra_copy[['cell_type', 'sm_name']])\n\n# Crear un DataFrame con las columnas codificadas\ndf_encoded = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(['cell_type', 'sm_name']))\n\n# Concatenar las columnas codificadas con el resto del DataFrame (excluyendo 'cell_type' y 'sm_name')\ndf_parqdtra_encoded = pd.concat([df_parqdtra_copy.drop(columns=['cell_type', 'sm_name']), df_encoded], axis=1)\n\nX = df_encoded\n# Las columnas de genes\ny = df_parqdtra_encoded[df_parqdtra.columns[4:]]\n"

Aca se toman los datos para el modelo de machine learning. Aplica One-Hot Encoding a las columnas categóricas 'cell_type' y 'sm_name', convirtiéndolas en variables numéricas, y luego las combina con el resto del DataFrame. Finalmente, coge estas columnas para las variables predictoras (X) y las columnas de genes como variables objetivo (y).

### Dividir el conjunto de datos

**se divide asi:**

In [5]:
'''print(X.head())
print("-------------------------------")
print(y.head())'''

'print(X.head())\nprint("-------------------------------")\nprint(y.head())'

```
            cell_type             sm_name sm_lincs_id  \
0            NK cells        Clotrimazole    LSM-5341   
1        T cells CD4+        Clotrimazole    LSM-5341   
2        T cells CD8+        Clotrimazole    LSM-5341   
3  T regulatory cells        Clotrimazole    LSM-5341   
4            NK cells  Mometasone Furoate    LSM-3349   

                                              SMILES  control      A1BG  \
0             Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1    False  0.104720   
1             Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1    False  0.915953   
2             Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1    False -0.387721   
3             Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1    False  0.232893   
4  C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...    False  4.290652   

   A1BG-AS1       A2M   A2M-AS1     A2MP1  ...      ZUP1      ZW10    ZWILCH  \
0 -0.077524 -1.625596 -0.144545  0.143555  ... -0.227781 -0.010752 -0.023881   
1 -0.884380  0.371834 -0.081677 -0.498266  ... -0.494985 -0.303419  0.304955   
2 -0.305378  0.567777  0.303895 -0.022653  ... -0.119422 -0.033608 -0.153123   
3  0.129029  0.336897  0.486946  0.767661  ...  0.451679  0.704643  0.015468   
4 -0.063864 -0.017443 -0.541154  0.570982  ...  0.758474  0.510762  0.607401   

      ZWINT      ZXDA      ZXDB      ZXDC    ZYG11B       ZYX     ZZEF1  
0  0.674536 -0.453068  0.005164 -0.094959  0.034127  0.221377  0.368755  
1 -0.333905 -0.315516 -0.369626 -0.095079  0.704780  1.096702 -0.869887  
2  0.183597 -0.555678 -1.494789 -0.213550  0.415768  0.078439 -0.259365  
...
4 -0.123059  0.214366  0.487838 -0.819775  0.112365 -0.122193  0.676629  

[5 rows x 18216 columns]
11184624
Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
   cell_type_Myeloid cells  cell_type_NK cells  cell_type_T cells CD4+  \
0                      0.0                 1.0                     0.0   
1                      0.0                 0.0                     1.0   
2                      0.0                 0.0                     0.0   
3                      0.0                 0.0                     0.0   
4                      0.0                 1.0                     0.0   

   cell_type_T cells CD8+  cell_type_T regulatory cells  \
0                     0.0                           0.0   
1                     0.0                           0.0   
2                     1.0                           0.0   
3                     0.0                           1.0   
4                     0.0                           0.0   

   sm_name_ABT-199 (GDC-0199)  sm_name_ABT737  \
0                         0.0             0.0   
1                         0.0             0.0   
2                         0.0             0.0   
3                         0.0             0.0   
4                         0.0             0.0   

   sm_name_AMD-070 (hydrochloride)  sm_name_AT 7867  sm_name_AT13387  ...  \
0                              0.0              0.0              0.0  ...   
1                              0.0              0.0              0.0  ...   
2                              0.0              0.0              0.0  ...   
...
3  0.865027  0.189114  0.224700 -0.048233  0.216139 -0.085024  
4  0.214366  0.487838 -0.819775  0.112365 -0.122193  0.676629  

[5 rows x 18212 columns]
```


**Análisis de Tablas:**

Aquí observamos cómo quedaron los DataFrames que vamos a utilizar en nuestro modelo. Además, se muestra cómo quedaron las variables `sm_names` y `cell_type`. Esto se hizo para poder realizar los modelos solicitados.


Utilicé la métrica **Mean Rowwise Root Mean Squared Error (MRRMSE)** para la evaluación y validación del modelo, ya que permite capturar el error promedio de manera más precisa a nivel de filas.

In [6]:
'''def MRRMSE(y_real, y_pred):
    rowwise_rmse = np.sqrt(np.mean((y_real - y_pred) ** 2, axis=1))
    return np.mean(rowwise_rmse)'''

'def MRRMSE(y_real, y_pred):\n    rowwise_rmse = np.sqrt(np.mean((y_real - y_pred) ** 2, axis=1))\n    return np.mean(rowwise_rmse)'

In [7]:
'''X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
resul = make_scorer(MRRMSE, greater_is_better = False)
print("Size of training set: {} size of validation set: {} size of test set: {}\n".
      format(X_train.shape[0], X_test.shape[0], X_test.shape[0]))'''

'X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)\nresul = make_scorer(MRRMSE, greater_is_better = False)\nprint("Size of training set: {} size of validation set: {} size of test set: {}\n".\n      format(X_train.shape[0], X_test.shape[0], X_test.shape[0]))'

Size of training set: 491 size of validation set: 123 size of test set: 123

In [8]:
'''print(X_test.head())
print("------------------------------")
print(y_test.head())'''

'print(X_test.head())\nprint("------------------------------")\nprint(y_test.head())'

```
    cell_type_Myeloid cells  cell_type_NK cells  cell_type_T cells CD4+  \
533                      0.0                 0.0                     0.0   
544                      0.0                 1.0                     0.0   
41                       0.0                 0.0                     1.0   
148                      0.0                 0.0                     0.0   
111                      0.0                 0.0                     1.0   

     cell_type_T cells CD8+  cell_type_T regulatory cells  \
533                     1.0                           0.0   
544                     0.0                           0.0   
41                      0.0                           0.0   
148                     1.0                           0.0   
111                     0.0                           0.0   

     sm_name_ABT-199 (GDC-0199)  sm_name_ABT737  \
533                         0.0             0.0   
544                         0.0             0.0   
41                          0.0             0.0   
148                         0.0             0.0   
111                         0.0             0.0   

     sm_name_AMD-070 (hydrochloride)  sm_name_AT 7867  sm_name_AT13387  ...  \
533                              0.0              0.0              0.0  ...   
544                              0.0              0.0              0.0  ...   
41                               0.0              0.0              0.0  ...   
...
148  0.394251  0.522069  0.251634  0.000173 -0.190295  1.289288  0.388646  
111  0.173986  1.400341  1.571638 -0.242477 -2.021973 -4.161092  0.953246  

[5 rows x 18212 columns]

```

In [9]:
'''print(X_train.head())
print("------------------------------")
print(y_train.head())'''

'print(X_train.head())\nprint("------------------------------")\nprint(y_train.head())'

```
     cell_type_Myeloid cells  cell_type_NK cells  cell_type_T cells CD4+  \
291                      0.0                 0.0                     0.0   
507                      0.0                 0.0                     0.0   
328                      0.0                 0.0                     1.0   
609                      0.0                 0.0                     0.0   
69                       0.0                 0.0                     1.0   

     cell_type_T cells CD8+  cell_type_T regulatory cells  \
291                     1.0                           0.0   
507                     1.0                           0.0   
328                     0.0                           0.0   
609                     0.0                           1.0   
69                      0.0                           0.0   

     sm_name_ABT-199 (GDC-0199)  sm_name_ABT737  \
291                         0.0             0.0   
507                         0.0             0.0   
328                         0.0             0.0   
609                         0.0             0.0   
69                          0.0             0.0   

     sm_name_AMD-070 (hydrochloride)  sm_name_AT 7867  sm_name_AT13387  ...  \
291                              0.0              0.0              0.0  ...   
507                              0.0              0.0              0.0  ...   
328                              0.0              0.0              0.0  ...   
...
609  1.073983  0.356939 -0.029603 -0.528817  0.105138  0.491015 -0.979951  
69   0.192888  0.179910  0.143050  0.220797 -0.881506 -1.374488 -0.258518  

[5 rows x 18212 columns]
```


En estas tablas se observa los datos utilizados para el modelo. Los datos se dividen en conjuntos de entrenamiento y prueba, denominados `X_train`, `X_test`, `y_train`, y `y_test`.

# Modelo de regresion KNN:

In [10]:
'''
pipeline_kr = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', KNeighborsRegressor())
])
param_grid_kr = {
    'regressor__n_neighbors': [5, 10],
    'regressor__weights': ['uniform', 'distance']
}

# Configurar validación cruzada
kfo = KFold(n_splits=3, shuffle=True, random_state=42)

# Crear el scorer utilizando la métrica Mean Rowwise Root Mean Squared Error (MRRMSE)
def MRRMSE(y_real, y_pred):
    rowwise_rmse = np.sqrt(np.mean((y_real - y_pred) ** 2, axis=1))
    return np.mean(rowwise_rmse)

resul = make_scorer(MRRMSE, greater_is_better=False)

# Inicializar GridSearchCV
grid_knr = GridSearchCV(pipeline_kr, param_grid_kr, cv=kfo, scoring=resul, n_jobs=-1)

try:
    # Ajustar el modelo
    grid_knr.fit(X_train, y_train)
except KeyboardInterrupt:
    print("Interrupción manual del proceso.")

# Realizar predicciones y mostrar los resultados
knn_y_pred = grid_knr.predict(X_test)
print(f'Best parameters: {grid_knr.best_params_}')
print(f'MRRMSE en el conjunto de prueba: {MRRMSE(y_test, knn_y_pred)}')
'''

'\npipeline_kr = Pipeline([\n    (\'scaler\', StandardScaler()),\n    (\'regressor\', KNeighborsRegressor())\n])\nparam_grid_kr = {\n    \'regressor__n_neighbors\': [5, 10],\n    \'regressor__weights\': [\'uniform\', \'distance\']\n}\n\n# Configurar validación cruzada\nkfo = KFold(n_splits=3, shuffle=True, random_state=42)\n\n# Crear el scorer utilizando la métrica Mean Rowwise Root Mean Squared Error (MRRMSE)\ndef MRRMSE(y_real, y_pred):\n    rowwise_rmse = np.sqrt(np.mean((y_real - y_pred) ** 2, axis=1))\n    return np.mean(rowwise_rmse)\n\nresul = make_scorer(MRRMSE, greater_is_better=False)\n\n# Inicializar GridSearchCV\ngrid_knr = GridSearchCV(pipeline_kr, param_grid_kr, cv=kfo, scoring=resul, n_jobs=-1)\n\ntry:\n    # Ajustar el modelo\n    grid_knr.fit(X_train, y_train)\nexcept KeyboardInterrupt:\n    print("Interrupción manual del proceso.")\n\n# Realizar predicciones y mostrar los resultados\nknn_y_pred = grid_knr.predict(X_test)\nprint(f\'Best parameters: {grid_knr.best_param

## LOS RESULTADOS K-NN:

```
Best parameters: {'regressor__n_neighbors': 10, 'regressor__weights': 'uniform'}
MRRMSE en el conjunto de prueba: 1.0797109579201776
```
*Se muestran los resultados de esta forma por el alto costo computacional.*

Los resultados muestran que el modelo KNN-Regressor, con los mejores parámetros identificados (`n_neighbors` = 10 y `weights` = 'uniform'), presenta un rendimiento moderado en el conjunto de prueba, con un MRRMSE de 1.08.

**Interpretación:**

1. **Número de Vecinos (`n_neighbors`)**: Con 10 vecinos, el modelo podría no estar capturando todas las variaciones en los datos. aumentar el numero de vecinos nos ayudaria a tener un mejor score aunque esto también puede incrementar el tiempo de cálculo y potencialmente llevar a un sobreajuste si se elige un número excesivamente alto.

2. **Pesos Uniformes (`weights` = 'uniform')**: El uso de pesos uniformes implica que todos los vecinos tienen el mismo peso en las predicciones.

3. **Capacidad de Generalización**: El MRRMSE relativamente alto sugiere que el modelo aún no está generalizando de manera óptima.

En resumen, aunque los parámetros seleccionados han mejorado el rendimiento, el MRRMSE indica que el modelo aún puede ser optimizado. Considerar ajustes adicionales en la configuración del KNN.



# Linear Regression

In [11]:
'''
linear = LinearRegression()

# Configurar el pipeline con un escalador y el modelo de regresión lineal
pipe = Pipeline([('scaler', StandardScaler()), ('lr', linear)])
pipe.fit(X_train, y_train)

# Predicciones
y_predtrain = pipe.predict(X_train)
y_pred = pipe.predict(X_test)

# Calcular MRRMSE
mrrmse_train = MRRMSE(y_train, y_predtrain)
mrrmse = MRRMSE(y_test, y_pred)

# Coeficientes e Intercepción
coefficients = pipe.named_steps['lr'].coef_
intercept = pipe.named_steps['lr'].intercept_

# Imprimir resultados
print(f'MRRMSE en el conjunto de entrenamiento: {mrrmse_train}')
print(f'Intercepción del modelo: {intercept}')
'''

"\nlinear = LinearRegression()\n\n# Configurar el pipeline con un escalador y el modelo de regresión lineal\npipe = Pipeline([('scaler', StandardScaler()), ('lr', linear)])\npipe.fit(X_train, y_train)\n\n# Predicciones\ny_predtrain = pipe.predict(X_train)\ny_pred = pipe.predict(X_test)\n\n# Calcular MRRMSE\nmrrmse_train = MRRMSE(y_train, y_predtrain)\nmrrmse = MRRMSE(y_test, y_pred)\n\n# Coeficientes e Intercepción\ncoefficients = pipe.named_steps['lr'].coef_\nintercept = pipe.named_steps['lr'].intercept_\n\n# Imprimir resultados\nprint(f'MRRMSE en el conjunto de entrenamiento: {mrrmse_train}')\nprint(f'Intercepción del modelo: {intercept}')\n"

## LOS RESULTADOS REGRESION LINEAL:

```
MRRMSE en el conjunto de entrenamiento: 0.9926474178039006
Intercepción del modelo: [ 0.02240326  0.36188023  0.26452489 ...  0.17108057 -0.17544219
 -0.09517982]
```
*Se muestran los resultados de esta forma por el alto costo computacional.*


Los resultados muestran que el modelo de regresión lineal tiene un **MRRMSE de 0.993** en el conjunto de entrenamiento, lo que indica un ajuste relativamente bueno a los datos de entrenamiento. Este valor es inferior a 1, sugiriendo que el modelo predice con una buena precisión.

La **intercepción del modelo** muestra una variedad de valores, con algunos positivos y otros negativos, reflejando cómo el modelo ajusta las predicciones en función de las características de entrada.

**Conclusión**: El modelo presenta un error de entrenamiento bajo, lo que es una señal positiva para el ajuste.

# Comparación de Resultados

## Evaluación de los Modelos

In [12]:
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error, r2_score
import pandas as pd

In [13]:
'''
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

# Realizar predicciones usando los modelos optimizados
knn_y_pred = grid_knr.predict(X_test)
lin_y_pred = pipe.predict(X_test)

# Crear un diccionario con las métricas de evaluación para ambos modelos
data = {
    'Modelo': ['K-NN', 'Linear Regression'],
    'MAPE': [
        mean_absolute_percentage_error(y_test, knn_y_pred),
        mean_absolute_percentage_error(y_test, lin_y_pred)
    ],
    'MAE': [
        mean_absolute_error(y_test, knn_y_pred),
        mean_absolute_error(y_test, lin_y_pred)
    ],
    'RMSE': [
        mean_squared_error(y_test, knn_y_pred, squared=False),  # RMSE = Raíz del MSE
        mean_squared_error(y_test, lin_y_pred, squared=False)
    ],
    'MSE': [
        mean_squared_error(y_test, knn_y_pred),
        mean_squared_error(y_test, lin_y_pred)
    ],
    'R²': [
        r2_score(y_test, knn_y_pred),
        r2_score(y_test, lin_y_pred)
    ]
}

# Convertir el diccionario en un DataFrame de Pandas para una mejor visualización
df = pd.DataFrame(data)
print(df)
'''

"\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)\n\n# Realizar predicciones usando los modelos optimizados\nknn_y_pred = grid_knr.predict(X_test)\nlin_y_pred = pipe.predict(X_test)\n\n# Crear un diccionario con las métricas de evaluación para ambos modelos\ndata = {\n    'Modelo': ['K-NN', 'Linear Regression'],\n    'MAPE': [\n        mean_absolute_percentage_error(y_test, knn_y_pred),\n        mean_absolute_percentage_error(y_test, lin_y_pred)\n    ],\n    'MAE': [\n        mean_absolute_error(y_test, knn_y_pred),\n        mean_absolute_error(y_test, lin_y_pred)\n    ],\n    'RMSE': [\n        mean_squared_error(y_test, knn_y_pred, squared=False),  # RMSE = Raíz del MSE\n        mean_squared_error(y_test, lin_y_pred, squared=False)\n    ],\n    'MSE': [\n        mean_squared_error(y_test, knn_y_pred),\n        mean_squared_error(y_test, lin_y_pred)\n    ],\n    'R²': [\n        r2_score(y_test, knn_y_pred),\n        r2_score(y_test, lin_y_pr

## COMPARACIÓN DE MODELOS:

```
              Modelo       MAPE       MAE      RMSE       MSE        R²
0               K-NN   8.585485  0.782599  1.544978  3.202212  0.213194
1  Linear Regression  13.028350  0.770418  1.432576  2.730580  0.295072
```
*Se muestran los resultados de esta forma por el alto costo computacional.*


Los resultados comparativos de los modelos muestran que el **modelo K-NN** tiene un **MAPE de 8.59**, un **MAE de 0.78**, un **RMSE de 1.54**, un **MSE de 3.20**, y un **R² de 0.21**. Estos valores indican que el modelo K-NN tiene una aproximacion relativamente baja en términos de precisión y capacidad de explicación de la variabilidad de los datos.

Por otro lado, la **regresión lineal** muestra un **MAPE de 13.03**, un **MAE de 0.77**, un **RMSE de 1.43**, un **MSE de 2.73**, y un **R² de 0.30**. Aunque el MAPE es mayor comparado con el K-NN, la regresión lineal tiene un menor MAE, RMSE y MSE, lo que indica que tiene ajuste relativamente más preciso.

**Conclusión**: Ambos modelos presentan tienes falencias en términos de ajuste y capacidad predictiva, pero la regresión lineal parece tener un desempeño superior en términos de precisión y capacidad de explicación de los datos comparado con el K-NN.